# Logging

You should now have a fairly good grasp of streaming data with LSL. So far we've only logged data temporarily using the system terminal output. In this part we'll take a quick look at how we can output data and timestamps to a CSV file. If you already are familiar with file operations in Python, you might not need this part.

## Writing to a CSV File in Python

The built-in `csv` library makes CSV operations really easy. Let's take a look at how we would write a row to a new file.

In [9]:
import csv

with open('./csv-example.csv', mode = 'w') as file:                     # Open our example file in write mode. If it doesn't exist, create it.
    writer = csv.writer(file, delimiter = ',')                          # Create a new writer object that can be used to write to our file. Values are separated with a comma (delimiter).
    writer.writerow(['column_1', 'column_2', 'column_3'])               # Write a row with three columns (an array with 3 values) to the file.

    file.close()                                                        # Close the file

There is one particularly notable thing about the code above. When we're opening the file (`open`), we specify the mode as `w`, for write. This will always create a new file, *even if one exists already*, meaning that **attempting to reopen a file in `w` mode will always erase its previous contents!** You can try this out by running the cell above multiple times.

Change the mode from `w` to `a` (append). What happens when you repeatedly run the cell?

You can also write multiple rows at a time, instead of calling `writerow` repeatedly:

In [10]:
# Define the rows we want to write. Each row is an array of values.
rows_to_write = [                                   
    ['col_1', 'col_2', 'col_3'],
    [1.23, 3.42, 6.6],
    [0.95, 2.98, 7.1]
]

with open('./csv-example.csv', mode = 'w') as file:                     # Create a new file (w) to save data to
    writer = csv.writer(file, delimiter = ',')                          
    writer.writerows(rows_to_write)                                     # Write our rows to the file

    file.close()                                                        

Writing multiple rows at a time is especially handy when you are trying to write a lot of data. This should reduce system overhead as the file does not need to be opened and closed repeatedly in quick succession.

## Logging LSL Data

We've already seen how to print data to the system output (terminal). With our newfound knowledge of CSV operations, we can easily write LSL data to a separate file.

Let's start by quickly wrapping our writing operation into a function that will take a timestamp, data type and data value as parameters and write them into a 3-column CSV file.

In [ ]:
import csv

def write_data(ts, type, value):
    with open('./csv-example-lsl.csv', mode = 'a') as file:                     
        writer = csv.writer(file, delimiter = ',')                          
        writer.writerow([ts, type, value])

Then, let's bring over some familiar-looking code from part 2. These functions create an outlet and an inlet, as well as help push and pull samples as streams in the LSL network. Note that we've added the pulled sampels and timestamp as a `return` value to `pull_random_sample`.

In [ ]:
from pylsl import StreamInfo, StreamOutlet, StreamInlet, resolve_stream
import time
import random

def create_outlet():
    name = 'workshop_outlet'                                                                
    type = 'single_stream'                                                                  
    n_of_channels = 1                                                                       
    samping_rate = 1                                                                        
    value_type = 'float32'                                                                  
    outlet_id = 'workshop_outlet_1234'                                                      

    stream = StreamInfo(name, type, n_of_channels, samping_rate, value_type, outlet_id)     
    outlet = StreamOutlet(stream)                                                           

    return outlet

def output_random_sample(target_outlet):
    random_sample = round(random.random(), 2)                                               
    print('Outputting random sample:', random_sample)                                       

    target_outlet.push_sample([random_sample])                                              

def create_inlet():
    stream = resolve_stream('source_id', 'workshop_outlet_1234')[0]                     
    inlet = StreamInlet(stream)                                                         
    return inlet

def pull_random_sample(source_inlet):
    samples, ts = source_inlet.pull_sample()
    print('Received sample:', samples[0], 'with timestamp', ts)
    return samples, ts

Finally, let's modify the demo code so that pulled samples are written to a file using our helper function we just defined (`write_data(ts, type, value)`). Note that our CSV file should probably contain headers for easy handling afterwards, so we'll initialize the file in `w` mode when the script is first run. You could also do this with our `write_data` function, but it's been set to open files in `a` mode, meaning that the file from the previous run would not be deleted. Of course, you could always parameterize the write mode for the `write_data` function.

In [ ]:
from threading import Thread
from IPython.display import clear_output

clear_output()

# Initialize the log file with headers!

with open('./csv-example-lsl.csv', mode = 'w') as file:             # Old file is DELETED when the script starts running  
    writer = csv.writer(file, delimiter = ',')                          
    writer.writerow(['ts', 'type', 'value'])                        # Headers for the three columns


run_time = 20                                                       # Samples will be generated and collected for 20 seconds

# Demo: Run the sampler & outlet in a separate CPU thread to circumvent Jupyter limitations
# You probably don't need/want to do this in any real-world application (not threading, just running the outlet & inlet in a single script)

def run_sampling(run_time):                              
    outlet = create_outlet()

    while run_time > 0:
        output_random_sample(outlet)
        time.sleep(1)                                               # Artifically sleep for 1s between samples (1Hz sampling rate)
        run_time -= 1                                               # n-1 seconds left for sampling
    
    print('Sample output complete!')

sampler = Thread(target = run_sampling, args = [run_time])          # Assign our outlet to a separate CPU thread
sampler.start()

# Pull samples from the network AND WRITE THEM TO A FILE

inlet = create_inlet()

while run_time > 0:
    samples, sample_ts = pull_random_sample(inlet)                      # Pull the sample(s) and timestamp from the outlet
    write_data(ts = sample_ts, type = 'rand_float', value = samples[0]) # Write our data to the file using our writing function.

    run_time -= 1                                                   # the run_time variable has separate instances for this main thread and the sampling thread so we have to deduct this here as well

inlet.close_stream()                                                # Close the inlet! This allows our program to finish. Otherwise, LSL will hang indefinitely waiting to reconnect. This is a great safety feature but for the demo a bit annoying...